# Merging & Cleaning Datasets: FBI Crime Data and gun laws with Census data

### Author: Lydia
#### Last edited by: Lydia, Marla
#### Last edited date: December 6, 2017

This notebook takes the FBI crime dataset with the full MSA codes for all msas, and gun laws for all states and merges in census variables. It also prepares the dataset for analysis by imputing missing predictor values.

**Inputs:**
    - df_crime_gunlaw_msacodes.csv
    - Census data
**Output:** 
    - merged_dataset.csv

In [1]:
#set up
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error


#### Merge FBI data with Census data

In [2]:
df_crime_gunlaws=pd.read_csv('cleaned data/df_crime_gunlaw_msacodes.csv').drop(['Unnamed: 0.1','Unnamed: 0' ], axis=1)

In [3]:
#print(list(df_crime_gunlaws))
print(df_crime_gunlaws.shape)

(3893, 152)


In [4]:
df_crime_gunlaws.head()

,MSA_name,Total,Estimated,Rate,MSA_original_name,MSA,year,Metropolitan Division Code,CSA Code,CBSA Title,...,universal,universalh,universalpermit,universalpermith,violent,violenth,violentpartial,waiting,waitingh,lawtotal
0,"Abilene, TX",6,6,3.7,"Abilene, TX",10180,2006,NaN,NaN,"Abilene, TX",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
1,"Albany, GA",11,11,6.5,"Albany, GA",10500,2006,NaN,NaN,"Albany, GA",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
2,"Albany-Schenectady-Troy, NY",18,18,2.1,"Albany-Schenectady-Troy, NY",10580,2006,NaN,104.0,"Albany-Schenectady-Troy, NY",...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,60.0
3,"Albuquerque, NM",72,72,8.9,"Albuquerque, NM",10740,2006,NaN,106.0,"Albuquerque, NM",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,"Alexandria, LA",10,10,7.1,"Alexandria, LA",10780,2006,NaN,NaN,"Alexandria, LA",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0


In [5]:
files = ['DP05', 'S0101', 'S0701', 'S1101', 'S1201', 'S1401', 'S1501', 'S1701', 'S1702', 'S1901', 'S1902', 
         'S1903', 'S2101', 'S2301', 'S2407', 'S2501']
census_dict = {}

for f in files:
    dftemp=pd.read_csv('census data/Cleaned data/%s.csv'%f).drop('GEOID', axis=1)
    census_dict[f]=dftemp



In [6]:
census_dict['DP05'].head()

,MSA,year,pct_hispanic,pct_white,pct_black,pct_indian,pct_asian,pct_hawaiian,pct_other,pct_mixed
0,10140,2006,6.480227,83.89093,0.571333,3.518795,1.660916,0.135499,0.65375,3.08855
1,10140,2007,6.800000,84.20000,0.400000,3.600000,1.100000,0.300000,0.20000,3.50000
2,10140,2008,7.500000,83.40000,0.800000,4.200000,1.200000,0.100000,0.00000,2.80000
3,10140,2009,7.900000,82.90000,0.900000,4.400000,1.400000,0.000000,0.20000,2.30000
4,10140,2010,8.700000,81.40000,1.000000,3.700000,1.800000,0.100000,0.00000,3.30000


In [7]:
files = ['S0101', 'S0701', 'S1101', 'S1201', 'S1401', 'S1501', 'S1701', 'S1702', 'S1901', 'S1902', 
         'S1903', 'S2101', 'S2301', 'S2407', 'S2501']
df_merged = pd.merge(df_crime_gunlaws, census_dict['DP05'], on=["MSA", 'year'])

for f in files:
    df_merged = pd.merge(df_merged, census_dict[f], on=["MSA", 'year'])


In [8]:
df_merged.head()

,MSA_name,Total,Estimated,Rate,MSA_original_name,MSA,year,Metropolitan Division Code,CSA Code,CBSA Title,...,unemployed20_64_disable,pct_disabled20_64,pct16_manuf,pct16_info,pct16_finance,pct16_prof,pct16_edhealth,num_hunits,num_hunits_owned,num_hunits_rented
0,"Abilene, TX",6,6,3.7,"Abilene, TX",10180,2006,NaN,NaN,"Abilene, TX",...,9.1,15.423708,3.344813,1.236906,7.788779,6.362462,28.728655,59850,39716,20134
1,"Albany, GA",11,11,6.5,"Albany, GA",10500,2006,NaN,NaN,"Albany, GA",...,15.5,18.360382,NaN,NaN,NaN,NaN,NaN,61158,34718,26440
2,"Albany-Schenectady-Troy, NY",18,18,2.1,"Albany-Schenectady-Troy, NY",10580,2006,NaN,104.0,"Albany-Schenectady-Troy, NY",...,12.0,11.752883,6.583495,2.571309,7.730132,9.671520,25.792726,337905,221077,116828
3,"Albuquerque, NM",72,72,8.9,"Albuquerque, NM",10740,2006,NaN,106.0,"Albuquerque, NM",...,11.5,12.442383,6.613994,2.752665,6.048520,13.249259,21.192484,318482,218800,99682
4,"Alexandria, LA",10,10,7.1,"Alexandria, LA",10780,2006,NaN,NaN,"Alexandria, LA",...,13.4,18.640030,8.171938,1.597395,4.640854,6.709060,26.969229,55546,37650,17896


In [9]:
print(df_merged.shape)
#print(list(df_merged))

(3893, 326)


### Clean up dataset

In [10]:
#destring
df_merged['Total'] = df_merged['Total'].str.replace(',', '')
df_merged['Estimated'] = df_merged['Estimated'].str.replace(',', '')

In [11]:
df_merged['Total'] = pd.to_numeric(df_merged.Total)
df_merged['Estimated'] = pd.to_numeric(df_merged.Estimated)

In [12]:
# drop some extraneous vars & vars with lots of missing vars
deletelist = ['Metropolitan Division Code', 'CSA Code', 'CBSA Title', 'Metropolitan/Micropolitan Statistical Area', 
 'Metropolitan Division Title', 'CSA Title', 'County/County Equivalent', 'FIPS State Code', 'FIPS County Code', 
 'Central/Outlying County', 'poor_indian', 'poor_asian', 'poor_hawaiian', 'poor_other',
 'poor_mixed', 'poor_hispanic', 'lfpr_black', 'employed_black', 'unemployed_black', 'lfpr_hispanic',
 'employed_hispanic','unemployed_hispanic', 'MSA_original_name']

for var in deletelist:
    del df_merged[var]
    
df_merged.shape

(3893, 303)

#### Imputing missing values using model-based method

In [13]:
missing_cols = df_merged.columns[df_merged.isnull().any()].values
missing_cols.shape

(201,)

In [14]:
df_imp = df_merged.copy()
df_nm = df_merged.drop(missing_cols, axis=1)
df_nm = df_nm.drop(['Estimated', 'Rate', 'state', 'MSA_name', 'MSA', 'year'], axis=1)
df_nm.shape

(3893, 96)

In [15]:
def impute_var(var, df_imp):
    # create dataframe including predictor with missing vals
    dftemp = pd.merge(df_nm, pd.DataFrame(data=df_imp[var]), left_index=True, right_index=True)
    dftemp = dftemp.dropna(axis=0, how='any')
    
    # select predictors using lasso first 
    X_nm = dftemp[dftemp.columns[0:-1]]
    y_nm = dftemp[var]
    
    lasso = LassoCV(cv=10, alphas=lambdas, fit_intercept=False, normalize=False)
    preprocessing.StandardScaler()
    lasso.fit(X_nm, y_nm)
    
    betas = np.absolute(lasso.coef_)
    nonzero = np.where(betas > 1e-10)

    predictors = []
    for x in nonzero[0]:
        predictors.append(X_nm.columns[x])
        
    # build a model to impute 
    X_imp = X_nm[predictors]
    y_imp = dftemp[var]

    regress = LinearRegression()
    regress.fit(X_imp,y_imp)
    y_hat = regress.predict(X_imp)
    
    # predict missing values using dataframe with missing Ys (df_nm2)
    X_missing = df_imp[df_imp[var].isnull()][predictors]
    y_missing = regress.predict(X_missing)
    y_missing_noise = y_missing + np.random.normal(loc=0, scale=np.sqrt(mean_squared_error(y_imp,y_hat)), size=y_missing.shape[0])
    y_missing_noise[y_missing_noise < 0] = 0
    
    missing_index = df_imp[var][df_imp[var].isnull()].index
    missing_series = pd.Series(data = y_missing_noise, index = missing_index)
    
    # fill in imputed values
    df_imp[var] = df_imp[var].fillna(missing_series)
    
    return df_imp

In [16]:
lambdas = [.05,1,5,10,50,100,500,1000]
for x in missing_cols:
    df_imp = impute_var(x, df_imp)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

...

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

In [17]:
df_imp[missing_cols].describe()

,Total,age18longgunpossess,age18longgunsale,age21handgunpossess,age21handgunsale,age21longgunpossess,age21longgunsale,age21longgunsaled,alcoholism,alctreatment,...,lfpr20_64_f,employed20_64_f,unemployed20_64_f,unemployed20_64_disable,pct_disabled20_64,pct16_manuf,pct16_info,pct16_finance,pct16_prof,pct16_edhealth
count,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,...,3893.000000,3893.000000,3893.00000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000,3893.000000
mean,44.993064,0.245258,0.491659,0.130491,0.255582,0.015776,0.003135,0.003111,0.175314,0.075803,...,72.073830,66.937749,6.91245,15.171358,11.625804,10.976347,1.880107,5.842909,9.352796,23.611093
std,105.443836,0.430157,0.499815,0.336886,0.436078,0.124284,0.055465,0.055463,0.380110,0.264671,...,5.078401,5.849052,2.86677,6.281888,3.212873,5.240442,0.696958,2.101069,2.826492,4.586749
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,42.700000,40.200000,0.30000,0.000000,3.707824,0.000000,0.000000,0.000000,1.007725,0.000000
25%,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,69.000000,63.500000,4.90000,11.000000,9.315629,7.238412,1.410978,4.510033,7.406744,20.688412
50%,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,72.300000,67.100000,6.50000,14.500000,11.310279,10.163135,1.811153,5.692966,8.996812,22.957453
75%,32.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,75.100000,70.300000,8.50000,19.000000,13.535088,14.133571,2.248709,6.995898,10.975624,25.987679
max,1092.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,87.500000,85.400000,22.60000,57.300000,39.084156,43.135350,5.174798,22.269805,21.943795,50.275941


In [18]:
df=df_imp.sort_values(by=['MSA_name'])

In [19]:
df.head()

,MSA_name,Total,Estimated,Rate,MSA,year,state,age18longgunpossess,age18longgunsale,age21handgunpossess,...,unemployed20_64_disable,pct_disabled20_64,pct16_manuf,pct16_info,pct16_finance,pct16_prof,pct16_edhealth,num_hunits,num_hunits_owned,num_hunits_rented
0,"Abilene, TX",6.0,6,3.7,10180,2006,Texas,0.0,1.0,0.0,...,9.1,15.423708,3.344813,1.236906,7.788779,6.362462,28.728655,59850,39716,20134
2078,"Abilene, TX",3.0,3,1.8,10180,2012,Texas,0.0,1.0,0.0,...,17.3,11.859934,6.174560,0.683455,5.184577,7.095610,24.451843,60066,35970,24096
2805,"Abilene, TX",10.0,10,5.9,10180,2014,Texas,0.0,1.0,0.0,...,8.7,13.773247,17.077829,0.717795,2.922261,9.927420,26.428519,59932,36872,23060
1723,"Abilene, TX",5.0,5,3.0,10180,2011,Texas,0.0,1.0,0.0,...,8.4,13.895194,4.893295,1.494660,4.510778,7.861076,26.143985,58974,39635,19339
1367,"Abilene, TX",5.0,5,3.1,10180,2010,Texas,0.0,1.0,0.0,...,12.9,14.616044,4.202828,1.534571,7.254727,7.825162,29.619518,59857,37501,22356


In [20]:
df.shape

(3893, 303)

In [21]:
df.to_csv('cleaned data/merged_dataset.csv')
